In [ ]:
#Section 1 ::: Importing libraries

import plotly as py # The library resource for the graph development
import plotly.graph_objs as go #This contains the graph object of the plotly like layout
import ipywidgets as widget #This library provide widget in iPython which had been used to make plotly graph interative
import numpy as np #Numpy has been used in the data cleaning for the data type conversion
import pandas as pd #Pandas has been used for the data file import
import math #This library resource provide mathematical functions which had been used in the data manupulation for graphs
import colorlover as cl # This qualitative data color palette had been used from this library
py.offline.init_notebook_mode(connected = True) # To run the plotly graphs offline
py.tools.set_credentials_file(username='yogeshgupta', api_key='wMOP1j6DGHTDPNsNkg8K') #Plotly setup(Not required for offline graphs)
from IPython.core.display import display, HTML, Image #For adding some HTML in the graph flow






#Section 2 ::: Data Cleaning 


#Importing Data from local csv files using Pandas
PopulationData = pd.read_csv("/Users/legion y/Desktop\Visualization on Organisation for Economic Co-operation and Development/Population93-18.csv")
DeathData=pd.read_csv("/Users/legion y/Desktop\Visualization on Organisation for Economic Co-operation and Development/HEALTH_STAT_01122018184305539.csv")
CountryData=pd.read_csv("/Users/legion y/Desktop\Visualization on Organisation for Economic Co-operation and Development/Country_List_ISO_3166_Codes_Latitude_Longitude.csv")

##0. Sorting data by Countries Name
PopulationData = PopulationData.sort_values(by = ['Country'])
CountryData = CountryData.sort_values(by = ['Country'])

##1.Checking for the missing Values in PopulationData
PopulationData.isnull().values.any()
DeathData.isnull().values.any()
CountryData.isnull().values.any()
####No Null Data found

##2.Data Type Check
PopulationData.dtypes
DeathData.dtypes
CountryData.dtypes
###Death Data Value type is different from the population data which will be used :: Need to update


##3.Data Scale Check and extra coloumn check
PopulationData.head()
DeathData.head()
###Value of the population is in thousands hence while performing actions it has to taken care.

##4.Data interconnectivity check
(DeathData['Country'][~DeathData.Country.isin(CountryData.Country)]).unique()
(DeathData['Country'][~DeathData.Country.isin(PopulationData.Country)]).unique()
(PopulationData['Country'][~PopulationData.Country.isin(CountryData.Country)]).unique()

##Fixit for DataType
DeathData['Value'] = DeathData['Value'].astype(np.float64)

##Fixit for Extra Column Check
PopulationData = PopulationData.drop(columns=['Flag Codes','Flags','Subject','SUB','YEA'])

#Fixit for interconnectivity
DeathData = DeathData[DeathData.Country.isin(CountryData.Country)]
DeathData = DeathData[DeathData.Country.isin(PopulationData.Country)]
PopulationData = PopulationData[PopulationData.Country.isin(CountryData.Country)]





##Section 3 :: Data Visualization
## This cell contains the code for the graphs
display(HTML("<center><h1>Statistics of Deaths by Several Diseases from OECD Data</center>"))

### Assignment of some useful variables
DeathCauses = DeathData.Variable.unique()
Countries = DeathData.Country.unique()
slope = 2.666051223553066e-05

##Here begins the Code for the map showing population aka Graph 1, Plotmap

#Personal Mapbox_Token need to change it while forking
#Get you own access token here 'https://www.mapbox.com/studio/'
mapbox_access_token = 'pk.eyJ1IjoiZ2F1cmF2c2hpdmhhcmUiLCJhIjoiY2pwcGw0amRsMGIwOTQybjAxeWVkb3VyayJ9.tbUz32On06X7-YOlTLfVwg'



hover_textP = []
bubble_sizeP = []

##for the customized hover text and bubble size
for index, row in PopulationData.iterrows():
    hover_textP.append(('Country: {Country}<br>'+
                       'Population: {Value} thousand').format(Country=row['Country'],
                                                        Value=row['Value']))
    bubble_sizeP.append(math.sqrt(row['Value']*slope))
    
PopulationData['Text'] = hover_textP
PopulationData['Size'] = bubble_sizeP

### Data trace for the map
datamap = [
    go.Scattermapbox(
        lat=CountryData['Latitude (average)'][CountryData.Country.isin(DeathData['Country'])],
        lon=CountryData['Longitude (average)'][CountryData.Country.isin(DeathData['Country'])],
        mode='markers',
        marker=dict(
            sizemode = 'area',
            sizeref = 20*max(DeathData['Value'])/100**5,
            size= PopulationData['Size'][(PopulationData['Year'] == 2015) & (PopulationData.Country.isin(DeathData['Country']))],
            color = '#ff6347',
        ),
        text=PopulationData['Text'][(PopulationData['Year'] == 2015) & (PopulationData.Country.isin(DeathData['Country']))],
    )
]

### Layout of the map
layoutmap = go.Layout(
    title= 'Population in the OECD Countries(2015)',
    font=dict(family='Gothic', size=18, color='#7f7f7f'),
    height = 600,
    width = 1000,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=5,
        center=dict(
            lat=10,
            lon=62
        ),
        pitch=0,
        zoom=0.7
    ),
)

## Figure definition for the map
figmap = dict(data=datamap, layout=layoutmap)
plotmap = py.offline.iplot(figmap)
##plotmap 










##Here begins the code for the Time Chart aka Graph2 PlotStack

### My color palette that will be using the below graphs
#my_color = ['#ff6347','#ffa600','#ff1493','#ffff00','#ff7c43','#f95d6a','#a05195','#665191','#2f4b7c','#003f5c','#191970']
#my_color = ['#191970','#003f5c','#2f4b7c','#665191','#a05195','#f95d6a','#ff7c43','#ffff00','#ff1493','#ff6347','#ffa600']
my_color = cl.scales['11']['qual']['Set3'] 


display(HTML("<Center><h1>Please choose a country"))
### Function works for the Country Widget  
def update_timeseries(country):
    timeseriesdata = []
    #Layout Definition for Slacked Graph(TimeSeries)
    timeseriesLayout = go.Layout(
        title = country[0] + ' Death Causes in the Years',
        #font=dict(family='Courier New, monospace', size=10, color='#7f7f7f'),
        images=[dict(
        source="https://user-images.githubusercontent.com/35273305/50044042-fbf7b780-0075-11e9-9e12-d8734e2ac1c8.png",
        xref="paper", yref="paper",
        x=0.1, y=1,
        sizex=0.2, sizey=0.2,
        xanchor="center", yanchor="bottom"
        )],
        xaxis = dict(
            title = 'Years',
            dtick = 2
        ),
        yaxis = dict(
            title = 'Percentage of People Died in Total Population',
            rangemode = 'tozero',
            ticksuffix = '%',

        ),
    )
    for A in country:
        for i in range(1,len(DeathCauses)):    
            #Defining the trace for the Slacked Graph
            trace = dict(
                x=DeathData['Year'][(DeathData['Country'] == A) & (DeathData['Variable'] == DeathCauses[0])],
                y=100 * ((DeathData['Value'][(DeathData['Country'] == A) & (DeathData['Variable'] == DeathCauses[i]) & (DeathData.Year.isin(PopulationData['Year'][(PopulationData['Country'] == A)]))])/(PopulationData['Value'][(PopulationData['Country'] == A) & (PopulationData.Year.isin(DeathData['Year'][(DeathData['Country'] == A) & (DeathData['Variable'] == DeathCauses[0])]))].values*1000)),
                hoverinfo='y',
                text = 'of total population',
                mode='lines',
                line=dict(width=0.5,shape = 'spline',color=my_color[i]),
                stackgroup='one',
                name = DeathCauses[i]
            )
            timeseriesdata.append(trace)

    timeseriesfig = dict(data=timeseriesdata,layout = timeseriesLayout)
    py.offline.iplot(timeseriesfig)
country = widget.SelectMultiple(options = Countries,value = ('Australia',), description = 'Country :',disabled = False)
widget.interactive(update_timeseries,country = country)
link2= widget.interactive(update_timeseries,country = country)
display(link2)



display(HTML("<Center><h1>Please choose Cause(s) and Year"))
#Plot for the Countries

#slope = 2.666051223553066e-05
hover_text = []
bubble_size = []

for index, row in DeathData.iterrows():
    hover_text.append(('Country: {Country}<br>'+
                      'Variable: {Variable}<br>'+
                      'Deaths: {Value}<br>').format(Country=row['Country'],
                                            Variable=row['Variable'],
                                            Value=row['Value']))
    bubble_size.append(math.sqrt(row['Value']*slope))


DeathData['Text'] = hover_text
DeathData['Size'] = bubble_size

#Interative graph
def update_plot(cause,Y):
    
    data = []
    for C in cause:
        #Trace for plot
        #print(Y)
        #print(data)
        trace = go.Scatter(
       #     y = ((DeathData['Value'][(DeathData['Country'] == Countries[0]) & (DeathData['Year'] == Y)])/(PopulationData['Value'][(PopulationData['Country'] == Countries[0]) & (PopulationData['Year'] == Y)]).values) * 100,
            y = 100 * (((DeathData['Value'][(DeathData['Variable'] == C) & (DeathData['Year'] == Y)])/ (DeathData['Value'][( (DeathData['Year'] == Y) & (DeathData['Variable'] == DeathCauses[0]))]).values)),
            x = DeathData['Country'][(DeathData['Variable'] == C) & (DeathData['Year'] == Y)],
            mode = 'markers',
            #hoverinfo = text,
            name = C,
            text = (DeathData['Text'][(DeathData['Variable'] == C) & (DeathData['Year'] == Y)]),
            marker=dict(
                #color = my_color,
                sizemode = 'area',
                sizeref = 5*max(DeathData['Value'])/100**5,
                size = DeathData['Size'][(DeathData['Variable'] == C) & (DeathData['Year'] == Y)],
                line = dict(
                            width = 2
                ),
            )
        )
        data.append(trace)
        #print(data)
    fig = go.Figure(data=data,layout=layout)
    py.offline.iplot(fig,filename = 'Plot1.html')
    #display(HTML("<object data=\""+'Plot1.html'+"\" width=\"650\" height=\"500\"></object>"+"\n"))
   # fichier_html_graphs.write("  <object data=\""+'Plot1.html'+"\" width=\"650\" height=\"500\"></object>"+"\n"+"Hello")
    
    
layout = go.Layout(
    title = 'Health in the World',
   # width = 1000,
   # height = 600,
    xaxis = dict(
     #   title = 'Causes of Death',
        automargin = True,
    ),
    yaxis = dict(
        title='Percentage of Total Death',
        tickmode = 'linear',
        dtick = 2,
        ticksuffix = '%'
        #autorange = False,
    ),
    showlegend = False,
    legend = dict(
        x= 0,
        y= 1.1,
        orientation = "h"
    ),
)



#Year Line
Year = widget.IntSlider(min = 1993, max = 2016,value = 2012, description = 'Year', layout = widget.Layout(width = '1000px',font=dict(size=1)))
#Year = widget.SelectMultiple(options = list(range(1993,2019)),value = (1999, ), description = 'Year', layout = widget.Layout(width = '140px',font=dict(size=1)))
#Year.style.button_width = '65px'
#display(Year)
#Year = widget.Select(options = list(range(1993,2019),value = 1999, description = 'Year', layout = widget.Layout(width = '1000px',font=dict(size=1)))

#Cause = widget.SelectMultiple(options = DeathCauses, description = 'Cause of Death'),#, layout = widget.Layout(width = '140px',font=dict(size=1)))
Cause = widget.SelectMultiple(options = DeathCauses[1:11],value = (DeathCauses[1], ), description = 'Cause(s):', layout = widget.Layout(width = '500px',description_width = '500px',marginleft = '100px',alignment = 'center'))



link = widget.interactive(update_plot,Y=Year,cause = Cause)
#fichier_html_graphs.write("</body></html>")
#print("CHECK YOUR DASHBOARD.html In the current directory")

display(link)